In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

client = MlflowClient()

# Use the search_experiments function to get a list of available experiments
experiments = client.search_experiments()
print(len(experiments))

# Display the list of available experiments
for experiment in experiments:
    print(f"Experiment Name: {experiment.name}, Experiment ID: {experiment.experiment_id}")

In [96]:
import pandas as pd
df = pd.read_csv('data/nyc-rolling-sales.csv')
df.head()
df.shape

(84548, 22)

In [97]:
df = df[['NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS AT PRESENT', 'ZIP CODE', 'TOTAL UNITS', 'GROSS SQUARE FEET', 'YEAR BUILT', 'SALE PRICE']]
df['SALE PRICE'] = pd.to_numeric(df['SALE PRICE'], errors='coerce')
df['GROSS SQUARE FEET'] = pd.to_numeric(df['GROSS SQUARE FEET'], errors='coerce')
df.rename(columns={'BUILDING CLASS CATEGORY': 'building_category',
                   'BUILDING CLASS AT PRESENT': 'building_class',
                   'ZIP CODE': 'zip_code',
                   'TOTAL UNITS': 'total_unit',
                   'GROSS SQUARE FEET': 'square_feet',
                   'YEAR BUILT': 'year_built',
                   'SALE PRICE': 'price'
                   }, inplace=True)
df = df.dropna()
df.shape

(48248, 8)

In [98]:
avg_price = df[df['price'] != 0]['price'].mean()
df['price'].replace(0, avg_price, inplace=True)
avg_total_unit = df[df['total_unit']!=0]['total_unit'].mean()
df['total_unit'].replace(0, avg_total_unit, inplace=True)
avg_square_feet = df[df['square_feet']!=0]['square_feet'].mean()
df['square_feet'].replace(0, avg_square_feet, inplace=True)
df = df[(df['zip_code']!=0) & (df['year_built']!=0)]
df = df[(df['price'] >=20000) & (df['price'] <=3000000)]
df.shape
# df.nunique()

(42383, 8)

In [99]:
# Import libraries
# !pip install plotly
import seaborn as sns
import matplotlib.pyplot as plt
import re
import plotly.express as px

fig = px.box(df, y='price',color_discrete_sequence=px.colors.sequential.Agsunset,
             width=600, height=500)
fig.update_layout(title_text='Box Plot of Price by Gearbox Type')
fig.show()

In [95]:
df.describe()

,zip_code,total_unit,square_feet,year_built,price
count,44801.00000,44801.000000,4.480100e+04,44801.000000,4.480100e+04
mean,11043.86786,2.490776,3.750530e+03,1945.598000,8.860743e+05
std,403.18159,16.939458,2.354926e+04,33.206822,5.393472e+05
min,10001.00000,1.000000,6.000000e+01,1800.000000,2.000000e+04
25%,10472.00000,1.000000,1.560000e+03,1920.000000,4.500000e+05
50%,11218.00000,2.000000,2.360000e+03,1938.000000,7.450000e+05
75%,11238.00000,3.000000,4.810983e+03,1962.000000,1.466445e+06
max,11694.00000,2261.000000,3.750565e+06,2017.000000,3.000000e+06
